# Problem Definition

Build a classifier to predict the dress type, given an image


# Data

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

Data is available here:
https://github.com/zalandoresearch/fashion-mnist

The following are the labels

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

Here is an example how the data looks like (*each class takes three-rows*):

![](fashion-mnist-sprite.png)


### Download data

In [ ]:
from keras.datasets import fashion_mnist

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### Model 1: CNN (plain vanilla)

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

In [5]:
batch_size = 128
num_classes = 10
epochs=2

In [6]:
# input image dimensions
img_rows, img_cols = 28, 28

In [7]:
K.image_data_format()

'channels_last'

In [8]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
flatten_1 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2769024   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [14]:
sgd = SGD(lr=0.01)

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

In [16]:
%%time 

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 63s 1ms/step - loss: 0.8242 - acc: 0.7316 - val_loss: 0.6082 - val_acc: 0.7839
Epoch 2/2
60000/60000 [==============================] - 62s 1ms/step - loss: 0.5360 - acc: 0.8095 - val_loss: 0.5394 - val_acc: 0.8089
CPU times: user 4min 34s, sys: 26.5 s, total: 5min
Wall time: 2min 5s


In [17]:
score = model.evaluate(x_test, y_test, verbose=0)

In [18]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.539354685831
Test accuracy: 0.8089


### Model 2: CNN 

In [19]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


In [21]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [22]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

In [23]:
%%time 

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 40s 661us/step - loss: 0.6202 - acc: 0.7800 - val_loss: 0.4868 - val_acc: 0.8265
Epoch 2/2
60000/60000 [==============================] - 39s 655us/step - loss: 0.4119 - acc: 0.8550 - val_loss: 0.4013 - val_acc: 0.8583
CPU times: user 2min 27s, sys: 15.8 s, total: 2min 42s
Wall time: 1min 19s


In [24]:
score = model.evaluate(x_test, y_test, verbose=0)

In [25]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.401335403848
Test accuracy: 0.8583


### Model 3: CNN 

In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [28]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [29]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

In [30]:
%%time 

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 231s 4ms/step - loss: 0.7382 - acc: 0.7258 - val_loss: 0.4634 - val_acc: 0.8293
Epoch 2/2
60000/60000 [==============================] - 253s 4ms/step - loss: 0.4768 - acc: 0.8272 - val_loss: 0.3903 - val_acc: 0.8606
CPU times: user 17min 55s, sys: 1min 54s, total: 19min 50s
Wall time: 8min 3s


In [31]:
score = model.evaluate(x_test, y_test, verbose=0)

In [32]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.390345384574
Test accuracy: 0.8606
